In [2]:
import os
import math
import torch
import pandas as pd

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from huggingface_hub import notebook_login
from datasets import Dataset

c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DATASET_PATH = "poem_dataset.csv"
df = pd.read_csv(DATASET_PATH)
df

,Unnamed: 0,title,content,source,url
0,0,NaN,Bạn xấu như chiếc bóng\nCứ bám riết theo anh\n...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,NaN,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,2,NaN,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,3,NaN,Chơi thân không có nghĩa\nKhông cãi nhau bao g...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,NaN,"Có thể buồn chút ít\nMột mình, không người yêu...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...,...
104,104,NaN,Chợ trưa ai đợi mẹ\nBến khuya ai đợi đò\nSông ...,NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
105,105,NaN,Người ấy đến rất say\ntừ cao trời bay xuống\nm...,Thu 84,https://www.thivien.net/Nguy%E1%BB%85n-Thu%E1%...
106,106,NaN,Tôi giật mình đánh rơi\nCon mèo con xuống giến...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/T%C3%B4-H%C3%A0/%C3%81...
107,107,NaN,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...


In [4]:
def split_content(content):
    samples = []
    
    poem_parts = content.split("\n\n")
    for poem_part in poem_parts:
        poem_in_lines = poem_part.split("\n")
        if len(poem_in_lines) == 4:
            samples.append(poem_in_lines)
            
    return samples

df["content"] = df["content"].apply(lambda x: split_content(x))
df

,Unnamed: 0,title,content,source,url
0,0,NaN,"[[Bạn xấu như chiếc bóng, Cứ bám riết theo anh...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,NaN,"[[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhi...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,2,NaN,"[[Chiều vừa xốp trên tay, Chợt nghe thoáng ong...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,3,NaN,"[[Chơi thân không có nghĩa, Không cãi nhau bao...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,NaN,"[[Có thể buồn chút ít, Một mình, không người y...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...,...
104,104,NaN,"[[Chợ trưa ai đợi mẹ, Bến khuya ai đợi đò, Sôn...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
105,105,NaN,"[[Người ấy đến rất say, từ cao trời bay xuống,...",Thu 84,https://www.thivien.net/Nguy%E1%BB%85n-Thu%E1%...
106,106,NaN,[],[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/T%C3%B4-H%C3%A0/%C3%81...
107,107,NaN,"[[Ôi, con sóng chết khô,, vật vờ trong bùn quá...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...


In [5]:
df_exploded = df.explode("content")
df_exploded.reset_index(drop=True, inplace=True)
df_exploded = df_exploded.dropna(subset=["content"])
df_exploded

,Unnamed: 0,title,content,source,url
0,0,NaN,"[Bạn xấu như chiếc bóng, Cứ bám riết theo anh,...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,0,NaN,"[Nhưng họ sẽ biến mất, Khi trời phủ mây đen, T...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,1,NaN,"[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhiề...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
3,1,NaN,"[Rồi thêm chút công việc, Cho ta làm hàng ngày...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,2,NaN,"[Chiều vừa xốp trên tay, Chợt nghe thoáng ong ...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
...,...,...,...,...,...
218,107,NaN,"[Ôi, con sóng chết khô,, vật vờ trong bùn quán...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
219,108,NaN,"[Áng dương không biết sầu , Nằm mãi ở trên cao...","Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
220,108,NaN,"[ Em ơi sao tàn nhẫn , Nỡ xa rời vòng tay , M...","Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
221,108,NaN,"[Đợi em những đêm dài , Không một lời đối đáp,...","Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...


In [6]:
df_exploded["content"] = df_exploded["content"].apply(lambda x: "\n".join(x))
df_exploded

,Unnamed: 0,title,content,source,url
0,0,NaN,Bạn xấu như chiếc bóng\nCứ bám riết theo anh\n...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,0,NaN,Nhưng họ sẽ biến mất\nKhi trời phủ mây đen\nTứ...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,1,NaN,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
3,1,NaN,Rồi thêm chút công việc\nCho ta làm hàng ngày\...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,2,NaN,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
...,...,...,...,...,...
218,107,NaN,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
219,108,NaN,Áng dương không biết sầu \nNằm mãi ở trên cao\...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
220,108,NaN,Em ơi sao tàn nhẫn \nNỡ xa rời vòng tay \nMộ...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
221,108,NaN,Đợi em những đêm dài \nKhông một lời đối đáp\n...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...


In [7]:
TEST_SIZE = 0.1
poem_dataset = Dataset.from_pandas(df_exploded)
poem_dataset = poem_dataset.train_test_split(test_size=TEST_SIZE)

In [8]:
MODEL_NAME = "danghuy1999/gpt2-viwiki"

tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
MAX_SEQ_LEN = 100

# Modified preprocess function to accept tokenizer as argument
def preprocess_function(row, tokenizer, max_seq_len=MAX_SEQ_LEN):
    return tokenizer(
        row["content"],
        max_length=max_seq_len,
        padding="max_length",
        truncation=True
    )

tokenized_poem_dataset = poem_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=poem_dataset["train"].column_names,
    fn_kwargs={'tokenizer': tokenizer}  # Pass tokenizer explicitly
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


Map (num_proc=4): 100%|██████████| 21/21 [00:05<00:00,  4.12 examples/s]


In [9]:
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

training_args = TrainingArguments(
    output_dir="gpt2_viet_poem_generation",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_poem_dataset["train"],
    eval_dataset=tokenized_poem_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()


C:\Users\Admin\AppData\Local\Temp\ipykernel_174520\4193928461.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=230, training_loss=5.998910389775816, metrics={'train_runtime': 50.0401, 'train_samples_per_second': 35.971, 'train_steps_per_second': 4.596, 'total_flos': 91860480000000.0, 'train_loss': 5.998910389775816, 'epoch': 10.0})

In [10]:
model.save_pretrained("gpt2_viet_poem_generation")
tokenizer.save_pretrained("gpt2_viet_poem_generation")

('gpt2_viet_poem_generation\\tokenizer_config.json',
 'gpt2_viet_poem_generation\\special_tokens_map.json',
 'gpt2_viet_poem_generation\\vocab.json',
 'gpt2_viet_poem_generation\\merges.txt',
 'gpt2_viet_poem_generation\\added_tokens.json')

In [11]:
prompt = "Học học nữa học mãi\n"
device = "cuda" if torch.cuda.is_available() else "cpu"
inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
outputs = model.generate(
    inputs,
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    repetition_penalty=1.2
)
results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
results = results[0]
print()
for line in results.split("\n"):
    print(line)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Học học nữa học mãi
Một học trò của ông
Cho đến trường, khi đang học
Hai câu chuyện cho chuyện
Rê khi nói chuyện, và khi ấy không hề
Chứt là chuyện, nên có ta nghĩ lòng!
Năm mươi phút, có ta
